In [ ]:
#connect to drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/feature_selection/heart"

/content/drive/MyDrive/feature_selection/heart


In [ ]:
import timeit
import time


In [ ]:
#import library
import pandas as pd
import numpy as np
import random
import math

STEP 1: READ DATA FROM CSV FILE AND CREATE FEATURES, LABEL

In [ ]:
df = pd.read_csv("heart.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [ ]:
X = df.drop(labels=['target'], axis=1)
y = df['target']

In [ ]:
start = timeit.default_timer()

STEP 2: TAKE MUTUAL INFORMATION FROM PRE-CALCULATED FILES

In [ ]:
#Read data from file csv
label = pd.read_csv("label.csv")
labl = label.to_numpy()
mutual_info = labl.reshape(-1)

cost1 = pd.read_csv("mutual.csv")
cost = cost1.to_numpy()

cond = pd.read_csv("mutual_con.csv")
conditional = cond.to_numpy()



STEP 3: LOCAL SEARCH STRATEGY

In [ ]:
#ranking according information gain
Ranking = []
for i in range(X.shape[1]):
    Ik = mutual_info[i]
    cij = 0
    for j in range(X.shape[1]):
      if i!=j:
        cij += -cost[i, j] + conditional[i, j]
    Ranking.append(Ik + cij)

In [ ]:
#Objective for each candidate
def fit1(sol):

  if sum(sol) == 0:
    return math.inf
  Rank1 = []
  for i in range(len(sol)):
     if sol[i] == 0:
       Rank1.append(Ranking[i])
  return sum(Rank1) + sum(sol)


In [ ]:
def min_info_feature(individual):
    indi = individual.copy()
    # Default value should be an integer, in case individual has length == 0
    lowest_feature = -math.inf
    lowest_value = math.inf

    for index in range(len(indi)):
      if indi[index] == 1:
          indi[index] = 0
          obj1_fit = fit1(indi)
          if obj1_fit < lowest_value:
            # high_index do not change when there are no better solution
            lowest_feature = index
            lowest_value = obj1_fit
          indi[index] = 1


    return lowest_feature

In [ ]:
def max_info_feature(individual):
    # Default value should be an integer, in case individual has length == 0
    indi = individual.copy()
    high_index = -math.inf
    high_value = math.inf

    for i in range(len(indi)):
      if indi[i] == 0:
        indi[i] = 1
        # Can update later to decrease from O(n) to O(1)
        obj_fit = fit1(indi)
        if obj_fit < high_value:
          # high_index do not change when there are no better solution
          high_index = i
          high_value = obj_fit
        indi[i] = 0

    return high_index

In [ ]:
def initialPopulation(d):
   while True:
      M = np.random.randint(low=0, high=2, size=d)

      if sum(M) >= 3:
        break
   return M

In [ ]:
def swapMove(solution=None, i: int = 0, j: int = -1):
    sol = solution.copy()

    if j == -1:
        # Change a single bit
        sol[i] = 1 - sol[i]
    else:
        # Swap 2 bit
        sol[i], sol[j] = sol[j], sol[i]

    return sol


In [ ]:
#Algorithmn fix feature

experiment_bonus = []
for z in range(30):
  start = timeit.default_timer()
  time.sleep(1)
  MAX_STEP_UNCHANGE = 50
  MAX_LOCK = 5
  FEATURE_SELECT_MAX = 0.3
  FEATURE_SELECT_MIN = 0.2

  feature_count = X.shape[1]
  feature_select_max_count = math.ceil(feature_count * FEATURE_SELECT_MAX)
  feature_select_min_count = math.ceil(feature_count * FEATURE_SELECT_MIN)
  # print("Feature selected max ", feature_select_max_count)
  # print("Feature selected min ", feature_select_min_count)
  temp = 0
  while temp <= 5:
      individual =  initialPopulation(X.shape[1])
      tabu_lock = np.zeros(feature_count, dtype=int)

    # individual =  initialPopulation(feature_count)

      current_solution = individual.copy()
      current_objective = fit1(current_solution)

      best_solution = current_solution.copy()
      best_objective = current_objective

      # print(individual)

      unchange_step_count = 0

      while unchange_step_count < MAX_STEP_UNCHANGE:
          # print('unchange_step_count:', unchange_step_count)
          # print('best_objective:', best_objective)

          # Decrease tabu lock
          boolean_mask = tabu_lock.copy()  # Used as condition to update tabu_lock
          np.subtract(tabu_lock, 1, out=tabu_lock, where=boolean_mask > 0)
          # print("Boolean_mask ", boolean_mask)
          # print("Tabu_lock ", tabu_lock)
          # print(tabu_lock)

          unchange_step_count += 1

          index_max = max_info_feature(current_solution)
          index_min = min_info_feature(current_solution)

          # print(current_solution)
          feature_current_count = sum(current_solution)
          # print(feature_current_count)

          moves = []

          # Check if there is valid index_max and index_min
          if index_max != -math.inf and feature_current_count < feature_select_max_count:
            moves.append((index_max, -1))
          if index_min != -math.inf and feature_current_count > feature_select_min_count:
            moves.append((index_min, -1))
          if index_max != -math.inf and index_min != -math.inf and feature_current_count < feature_select_max_count and feature_current_count > feature_select_min_count:
            moves.append((index_max, index_min))

          # print(moves)

          candidates = [swapMove(current_solution, move[0], move[1]) for move in moves]
          moves_fitness = [fit1(candidate) for candidate in candidates]

          # Sort candidates by move_fitness
          moves = [move for _, move in sorted(zip(moves_fitness, moves))]
          candidates = [candidate for _, candidate in sorted(zip(moves_fitness, candidates))]
          moves_fitness = sorted(moves_fitness)
          # print("Moves ", moves)
          # print("Candidates ", candidates)
          # print("Move fitness ", moves_fitness)
          for i in range(len(moves)):
              # select the move with best objective value in neighbor
              select_move = moves[i]
              select_candidate = candidates[i]
              select_fitness = moves_fitness[i]
              # print("\nMove best: ", best_move)

              # Not tabu
              if tabu_lock[select_move[0]] == 0 and (select_move[1] == -1 or tabu_lock[select_move[1]] == 0):
                  current_solution = select_candidate.copy()
                  current_objective = select_fitness

                  if select_fitness < best_objective:
                      unchange_step_count = 0

                      best_solution = current_solution.copy()
                      best_objective = current_objective

                  # Lock feature
                  if select_move[0] != -1:
                    tabu_lock[select_move[0]] = MAX_LOCK
                  if select_move[1] != -1:
                    tabu_lock[select_move[1]] = MAX_LOCK

                  break

              else:
                  # Aspiration
                  if select_fitness < best_objective:
                      current_solution = select_candidate.copy()
                      current_objective = select_fitness

                      unchange_step_count = 0

                      best_solution = current_solution.copy()
                      best_objective = current_objective

                      # Lock feature
                      if select_move[0] != -1:
                        tabu_lock[select_move[0]] = MAX_LOCK
                      if select_move[1] != -1:
                        tabu_lock[select_move[1]] = MAX_LOCK

                      break
                  else:
                      pass
      temp+=1

  print('----------------------------------------------')
  print("Best found Solution: {} , Objvalue: {}".format(best_solution, best_objective))
  stop = timeit.default_timer()

  print('Time: {}'.format(stop - start))
  elapse = stop - start
  experiment_bonus.append(elapse)


In [ ]:
# import timeit
# start = timeit.default_timer()

# # Complete Search (only for small dataset)
# feature_count = X.shape[1]
# individual = np.ones(feature_count, dtype=int)

# best_solution = individual.copy()
# best_objective = fit1(best_solution)
# current_solution = best_solution.copy()
# current_objvalue = best_objective
# subset_count = 0

# def Try(k: int = 0):
#   global best_solution
#   global best_objective
#   global current_solution
#   global current_objvalue
#   global subset_count

#   if k == feature_count:
#     subset_count += 1
#     if sum(current_solution) == 0:
#       current_objvalue = math.inf
#     else:
#       current_objvalue = fit1(current_solution)

#     if current_objvalue < best_objective:
#       best_solution = current_solution.copy()
#       best_objective = current_objvalue

#   else:
#     current_solution[k] = 0
#     Try(k+1)
#     current_solution[k] = 1
#     Try(k+1)

#   return best_solution

# Try()

# print("Best found Solution: {} , Objvalue: {}".format(best_solution, best_objective))
# print("Search on {} subsets".format(subset_count))

# # Show time for complete search
# end = timeit.default_timer()
# print('Time: {}'.format(end - start))